In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

house_df = pd.read_csv('kc_house_data.csv')

In [ ]:
# something is bugged
# this can't be together with imports: https://github.com/jupyter/notebook/issues/3385
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300.0 # default: 72.0
mpl.rcParams['figure.figsize'] = [8., 4.5] # default: [6., 4.]

## Exercise 1
a) Create and train simple regression model of sqft_living to price.  
b) Evaluate and visualise created model.  
c) Create and plot higher order polynomial regression models.

In [ ]:
#a)

x = house_df.sqft_living.values.reshape(-1,1)[:1000]
y = house_df.price.values.reshape(-1)[:1000]

model = LinearRegression()

#Your code:
model.fit(x, y)

#Hint: .fit()

In [ ]:
#b)

#Your code:

y_pred = model.predict(x)

plt.scatter(x, y)
plt.plot(x, y_pred, color = 'red')
plt.show()


print(model.score(x, y))
#print(model.intercept_)

#Hint: model.predict(), model.intercept_,  model.coef_, model.score()

In [ ]:
#c)

import operator

scores = [[], []]

for order in range(1,9):
    plt.subplot(4, 2, order)

    poly = PolynomialFeatures(order, include_bias = False)
    x_order = poly.fit_transform(x)

    model = LinearRegression()

    model.fit(x_order, y)
    y_pred = model.predict(x_order)

    x_tmp, y_tmp = zip(*sorted(zip(x, y_pred), key=operator.itemgetter(0)))

    scores[0].append(order)
    scores[1].append(mean_squared_error(y, y_pred))
    
    plt.scatter(x, y)
    plt.plot(x_tmp, y_tmp, color='red')

plt.gcf().set_size_inches(8, 4*3)
plt.show()

    


In [ ]:
#Your code:

plt.plot(scores[0], scores[1])
plt.show()

## Exercise 2
In reality, we are not given the full dataset, so we need to create models which are able to handle unknown data.  
a) Split your data into training sample and test sample and repeat exercise 1 c). Plot training and test errors for all models. 

In [ ]:
#a)

scores = [[], [], []]

for order in range(1,9):
        
    poly = PolynomialFeatures(order, include_bias = False)
    x_order = poly.fit_transform(x)
    
    x_train, x_test, y_train, y_test = train_test_split(x_order, y, test_size=0.3, random_state=44)
    
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    y_train_pred = model.predict(x_train)

    scores[0].append(order)
    scores[1].append(mean_squared_error(y_test, y_pred))
    scores[2].append(mean_squared_error(y_train, y_train_pred))
    

plt.plot(scores[0], scores[1], label='Test error')
plt.plot(scores[0], scores[2], label='Train error')
plt.legend()
plt.show()
    
    

## Exercise 3
a) Train model to predict sqft_living for a given house price. Plot predictions.  
b) Apply more features and propose better model for predicting house prices. Try to beat proposed one.

In [ ]:
#a)

x = house_df.price.values.reshape(-1,1)[:1000]
y = house_df.sqft_living.values.reshape(-1)[:1000]

#Your code:

model = LinearRegression()
model.fit(x, y)

y_pred = model.predict(x)

plt.scatter(x, y)
plt.plot(x, y_pred, color='red')
plt.show()


In [ ]:
#b) Let's propose model, which uses living sqft, lot sqft, grade, view and all of their interactions up to third degree

X = house_df[['sqft_living', 'sqft_lot', 'grade', 'view']].values.reshape(-1,4)
y = house_df.price.values.reshape(-1)

poly = PolynomialFeatures(3, include_bias = False)
X = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=43)
    
model = LinearRegression() 
_= model.fit(X_train, y_train) 

y_pred = model.predict(X_test)
print("Mean squared error equals: {0}".format(mean_squared_error(y_pred, y_test)))

#Your model:

error = mean_squared_error(y_pred, y_test)

features = [
    'bathrooms',
    'sqft_above',
    'sqft_living15',
    'sqft_lot',
    'grade',
    'lat',
    'long'
]
x = house_df[features].values.reshape(-1, len(features))
y = house_df.price.values.reshape(-1)

for order in range(1,4):
    poly = PolynomialFeatures(order, include_bias = False)
    x_transformed = poly.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x_transformed,
        y,
        test_size=0.2, # test_size=0.5,
        random_state=43 # random_state=47
        # 75% # 75%
    )

    model = LinearRegression()
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    current_error = mean_squared_error(y_pred, y_test)
    print("Order: {1}, Mean squared error equals: {0} ({2:.0f}%)".format(
            current_error,
            order,
            current_error / error * 100
        )
    )